# Imports

In [ ]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import soft_lts, experiment, data_simulator, custom_wrapping
from experiment import Model, ParamGrid

logging.basicConfig(format='%(asctime)s - %(name)15s - %(levelname)8s - %(message)s', level=logging.INFO, datefmt="%Y-%m-%d %H:%M:%S")


%matplotlib inline 
# increase number of distinct colors
from cycler import cycler
plt.rcParams['axes.prop_cycle'] = cycler('color', [plt.get_cmap('tab10')(i) for i in range(20)])

# Experiment

In [ ]:
param_grids = [
    ParamGrid(name='OLS', model=Model.OLS),
    ParamGrid(
        name='SoftLTS single start - no smart init', model=Model.SOFT_LTS, smart_init=False,
        fit_params={'apply_c_steps': False, 'max_iter': 300},
        grid_params={
          'alpha': [0.5], 'regularization_strength': [0.0001], 'n_initial_subsets': [1], 'n_best_models': [1], 'n_initial_iters': [1]
    }),
    ParamGrid(
        name='SoftLTS 20 starts - no smart init', model=Model.SOFT_LTS, smart_init=False,
        fit_params={'apply_c_steps': False, 'max_iter': 300},
        grid_params={
          'alpha': [0.5], 'regularization_strength': [0.0001], 'n_initial_subsets': [20], 'n_best_models': [10], 'n_initial_iters': [10]
    }),
    ParamGrid(
        name='SoftLTS single start - xy wrap smart init', model=Model.SOFT_LTS, smart_init=True,
        fit_params={'apply_c_steps': False, 'max_iter': 300},
        grid_params={
          'alpha': [0.5], 'regularization_strength': [0.0001], 'n_initial_subsets': [1], 'n_best_models': [1], 'n_initial_iters': [1]
    }),
    ParamGrid(
        name='SoftLMS single start - no smart init', model=Model.SOFT_LMS, smart_init=False,
        fit_params={'apply_c_steps': False, 'max_iter': 300, 'normalize_gradients': True},
        grid_params={
          'alpha': [0.5], 'regularization_strength': [0.0001], 'n_initial_subsets': [1],  'n_best_models': [1],'n_initial_iters': [1]
    }),
    ParamGrid(
        name='SoftLMS 20 starts - no smart init', model=Model.SOFT_LMS, smart_init=False,
        fit_params={'apply_c_steps': False, 'max_iter': 300, 'normalize_gradients': True},
        grid_params={
          'alpha': [0.5], 'regularization_strength': [0.0001], 'n_initial_subsets': [20],  'n_best_models': [10],'n_initial_iters': [10]
    }),
    ParamGrid(
        name='SoftLMS single start - xy wrap smart init', model=Model.SOFT_LMS, smart_init=True,
        fit_params={'apply_c_steps': False, 'max_iter': 300, 'normalize_gradients': True},
        grid_params={
          'alpha': [0.5], 'regularization_strength': [0.0001], 'n_initial_subsets': [1],  'n_best_models': [1],'n_initial_iters': [1]
    }),
    ParamGrid(
        name='FastLTS 500 starts', model=Model.FAST_LTS, smart_init=False, fit_params={},
        grid_params={'alpha': [0.5], 'n_initial_subsets': [500], 'n_best_models': [10], 'n_initial_c_steps': [2]}
    ),
    ParamGrid(
        name='FastLTS single start - no smart init', model=Model.FAST_LTS, smart_init=False, fit_params={},
        grid_params={'alpha': [0.5], 'n_initial_subsets': [1], 'n_best_models': [1], 'n_initial_c_steps': [1]}
    ),
    ParamGrid(
        name='FastLTS single start - xy wrap smart init', model=Model.FAST_LTS, smart_init=True, fit_params={},
        grid_params={'alpha': [0.5], 'n_initial_subsets': [1], 'n_best_models': [1], 'n_initial_c_steps': [1]}
    ),
     ParamGrid(
        name='Classic LMS 3000 subsets', model=Model.LMS, smart_init=False, fit_params={}, grid_params={'nr_of_subsamples': [3000]}
    ),
    ParamGrid(name='MM estimator', model=Model.MM_ESTIMATOR)
]

In [ ]:
path = './Output/experiments_paper_final.pkl'

if os.path.exists(path):
    exp = experiment.Experiment.load(path)
else:
    exp = experiment.Experiment(
        scaler=StandardScaler,
        datasets=['bodyfat', 'ale', 'power', 'hardware', 'simulated_simple', 'simulated_a09'],
        simple_simulator_params={'contamination_distance_y': 5, 'contamination_distance_x': 5, 'bad_leverage_perc': 0.5},
        a09_simulator_params={'noise_std': 0.1, 'contamination_distance': 5, 'contamination_vertical_shift': -5, 'bad_leverage_perc': 0.5},
        param_grids=param_grids,
        repetitions=10,
        leverage_perc=0.5
    )

exp.run(store_intermediate=True, path=path, disable_gridsearch=True)
exp.save(path)

# Plots

In [ ]:
exp = experiment.Experiment.load('./Output/experiments_paper_final.pkl')

In [ ]:
datasets_real = {
    'bodyfat': 'Bodyfat',
    'hardware': 'Computer Hardware',
    'power': 'CCPP',
    'ale': 'Average Localization Error',
}

datasets_simul = {
    'simulated_simple': 'Simulated data: Uncorrelated features',
    'simulated_a09': 'Simulated data: Correlated features',
}


## Fast LTS vs Soft LTS

In [ ]:
models = {
    'OLS': 'OLS',
    'MM estimator': 'MM estimator',
    'S estimator': 'S estimator',
    'FastLTS 500 starts' : 'FastLTS 500 starts',
    'FastLTS single start - no smart init': 'FastLTS single start - random init',
    'FastLTS single start - xy wrap smart init': 'FastLTS single start - xy wrap init',
    'SoftLTS single start - no smart init': 'SoftLTS single start - random init',
    'SoftLTS single start - xy wrap smart init': 'SoftLTS single start - xy wrap init',
    'SoftLTS 20 starts - no smart init': 'SoftLTS 20 starts'
}

### Simulation data

In [ ]:
fig, axd = exp.plot_r2_and_fit_time(figsize=(22, 10),
    dataset_dict=datasets_simul,
    datasets=list(datasets_simul.keys()),
    models=models,
    y_label='Mean $R^2$ on test set',
    x_label='Outlier percentage',
    n_legend_col=4
)

### Real data

In [ ]:
fig, axd = exp.plot_r2_and_fit_time(figsize=(25, 13),
    dataset_dict=datasets_real,
    datasets=list(datasets_real.keys()),
    models=models,
    y_label='Mean $R^2$ on test set',
    x_label='Outlier percentage',
)

## Fast LMS vs Soft LMS

In [ ]:
models = {
    'OLS': 'OLS',
    # 'S estimator': 'S estimator',
    'MM estimator': 'MM estimator',
    'SoftLMS single start - no smart init' : 'SoftLMS single start - random init',
    'SoftLMS single start - xy wrap smart init': 'SoftLMS single start - xy wrap init',
    'SoftLMS 20 starts - no smart init': 'SoftLMS 20 starts',
    'Classic LMS 3000 subsets': 'Classic LMS 3000 subsets'
}

### Simulation data

In [ ]:
fig, axd = exp.plot_r2_and_fit_time(figsize=(22, 10),
    dataset_dict=datasets_simul,
    datasets=list(datasets_simul.keys()),
    models=models,
    y_label='Mean $R^2$ on test set',
    x_label='Outlier percentage',
    n_legend_col=3
)

### Real data

In [ ]:
fig, axd = exp.plot_r2_and_fit_time(figsize=(22, 13),
    dataset_dict=datasets_real,
    datasets=list(datasets_real.keys()),
    models=models,
    y_label='Mean $R^2$ on test set',
    x_label='Outlier percentage',
    n_legend_col=3
)


## Non-linear regression

### Data

In [ ]:
from scipy.stats import chi2

np.random.seed(42)
n_clean = 800
n_outlier = 200
n_observations = n_clean + n_outlier

row_idx = np.array([0, 1]).reshape(-1, 1)
column_idx = np.array([0, 1]).reshape(1, -1)
cov = np.power(0.9, np.abs(row_idx - column_idx))
X_clean = np.random.multivariate_normal(mean=np.array([0, 0]), cov=cov, size=n_clean)

eigenvalues, eigenvectors = np.linalg.eig(cov)
selected_direction = eigenvectors[:, np.argmin(eigenvalues)]
mahalanobis_distance = (
    (selected_direction).reshape(1, -1) @ np.linalg.inv(cov) @ (selected_direction).reshape(-1, 1)
)[0][0]
scaled_direction = selected_direction / np.sqrt(mahalanobis_distance)
mu_contaminated = scaled_direction * np.sqrt(chi2.ppf(0.975, 1))
X_outlier = np.random.multivariate_normal(mean=mu_contaminated, cov=cov * 1, size=n_outlier)

X_data = np.concatenate((X_clean, X_outlier))

augmented_X = np.c_[np.ones(n_observations), X_data]
y_data = np.sin(augmented_X @ np.array([1, 1, 1]).reshape(-1, 1))
gaussian_noise = np.random.normal(loc=0, scale=0.1, size=(n_observations, 1))
y_data += gaussian_noise
y_data[-200:] += 10


fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(121, projection='3d')
ax.scatter3D(X_data[:800, 0], X_data[:800, 1], y_data[:800], label='clean data')
ax.scatter3D(X_data[800:, 0], X_data[800:, 1], y_data[800:], label='outliers')
ax.set_zlabel('y', fontsize=15)
ax.set_xlabel('$X_1$', fontsize=15, fontweight='bold')
ax.set_ylabel('$X_2$', fontsize=15)

ax = fig.add_subplot(122, projection='3d')
ax.scatter3D(X_data[:800, 0], X_data[:800, 1], y_data[:800], label='Clean data')
ax.scatter3D(X_data[800:, 0], X_data[800:, 1], y_data[800:], label='Outliers')
ax.set_zlabel('y', fontsize=15)
ax.set_xlabel('$X_1$', fontsize=15)
ax.set_ylabel('$X_2$', fontsize=15)
ax.set_zticklabels([])
fig.legend(*ax.get_legend_handles_labels(), loc='upper center', fontsize=12)

### Experiment

In [ ]:
import keras, joblib
from keras.models import Sequential
from keras.layers import Dense, Layer
from keras.optimizers import Adam
from sklearn.metrics import r2_score

np.random.seed(42)
keras.utils.set_random_seed(42)
results = {}

y_wrap = custom_wrapping.wrap(y_data, rescale=True)
mlp_init = Sequential([
    Dense(units=100, activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=1, activation='linear')
])
mlp_init.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

history = mlp_init.fit(X_data, y_wrap, epochs=2000, batch_size=50, verbose=False)
init_weights = mlp_init.get_weights()

print('MSE')
mlp = Sequential([
    Dense(units=100, activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=1, activation='linear')
])
mlp.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))

history = mlp.fit(X_data, y_data, epochs=2000, batch_size=50, verbose=False)

results['mse'] = {
    'predictions' : mlp.predict(X_data),
    'r2': r2_score(y_data[:800], mlp.predict(X_data[:800])),
    'history': history.history['loss']
}

print('MSE init')
mlp.set_weights(init_weights)
history = mlp.fit(X_data, y_data, epochs=2000, batch_size=50, verbose=False)

results['mse_init'] = {
    'predictions' : mlp.predict(X_data),
    'r2': r2_score(y_data[:800], mlp.predict(X_data[:800])),
    'history': history.history['loss']
}

print('MAE')
mlp = Sequential([
    Dense(units=100, activation='relu'),
    Dense(units=100, activation='relu'),
    Dense(units=1, activation='linear')
])
mlp.compile(loss='mean_absolute_error', optimizer=Adam(learning_rate=0.001))

history = mlp.fit(X_data, y_data, epochs=2000, batch_size=50, verbose=False)

results['mae'] = {
    'predictions' : mlp.predict(X_data),
    'r2': r2_score(y_data[:800], mlp.predict(X_data[:800])),
    'history': history.history['loss']
}

print('MAE init')
mlp.set_weights(init_weights)
history = mlp.fit(X_data, y_data, epochs=2000, batch_size=50, verbose=False)

results['mae_init'] = {
    'predictions' : mlp.predict(X_data),
    'r2': r2_score(y_data[:800], mlp.predict(X_data[:800])),
    'history': history.history['loss']
}

print('Soft')
mlp = soft_lts.SoftLTS_MLP(
    layers=[
        Dense(100, activation='relu'),
        Dense(100, activation='relu'),
        Dense(1, activation='linear')
    ],
    learning_rate=0.01,
    alpha=0.7
)
mlp.fit(X_data, y_data, epochs=10000, verbosity=logging.ERROR, initial_weights=None)
results['soft'] = {
    'predictions' : mlp.predict(X_data),
    'r2': r2_score(y_data[:800], mlp.predict(X_data[:800])),
    'history': mlp.history
}

print('Soft init')
mlp = soft_lts.SoftLTS_MLP(
    layers=[
        Dense(100, activation='relu'),
        Dense(100, activation='relu'),
        Dense(1, activation='linear')
    ],
    learning_rate=0.01,
    alpha=0.7
)
mlp.fit(X_data, y_data, epochs=10000, verbosity=logging.ERROR, initial_weights=mlp_init.get_weights())

results['soft_init'] = {
    'predictions' : mlp.predict(X_data),
    'r2': r2_score(y_data[:800], mlp.predict(X_data[:800])),
    'history': mlp.history
}

joblib.dump(results, './Output/non-linear-experiment.pkl')

In [ ]:
%matplotlib inline
results = joblib.load('./Output/non-linear-experiment.pkl')
fig = plt.figure(figsize=(30, 20))
models = {
    'mse': 'MSE loss',
    'mae': 'MAE loss',
    'soft': 'Soft LTS loss',
    'mse_init': 'MSE loss with y-wrap initialisation',
    'mae_init': 'MAE loss with y-wrap initialisation',
    'soft_init': 'Soft LTS loss with y-wrap initialisation'
}
for i, model in enumerate(models, start=1):
    ax = fig.add_subplot(f'23{i}', projection='3d')
    ax.scatter3D(X_data[:800, 0], X_data[:800, 1], y_data[:800], label='clean data', alpha=0.5, s=100)
    ax.scatter3D(X_data[800:, 0], X_data[800:, 1], y_data[800:], label='outliers', alpha=0.5, s=100)
    ax.scatter3D(X_data[:, 0], X_data[:, 1], results[model]['predictions'], label='predictions', alpha=0.5, s=100)
    ax.set_zlabel('y', fontsize=30)
    ax.set_xlabel('$X_1$', fontsize=30)
    ax.set_ylabel('$X_2$', fontsize=30)
    ax.set_zticklabels([])
    ax.set_title(f'{models[model]} \n $R^2$ = {results[model]["r2"]:.2f}', size=30)
    ax.tick_params(axis='both', labelsize=20)
fig.legend(*ax.get_legend_handles_labels(), loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.05), fontsize=20, markerscale=3)
fig.canvas.draw()
fig.tight_layout()
plt.subplots_adjust(wspace=0, hspace=0.15)
    

In [ ]:
window = 10
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
ax.plot(pd.Series(results['soft']['history']).rolling(window=window).mean(), label='Soft LTS loss')
ax.plot(pd.Series(results['soft_init']['history']).rolling(window=window).mean(), label = 'Soft LTS loss with y-wrap initialisation')
ax.set_ylim((0, np.quantile(np.concatenate((results['soft']['history'], results['soft_init']['history'])), 0.995)))
ax.legend()
ax.set_title('(Smoothed) Training loss history')
ax.set_xlabel('Iteration')
ax.set_ylabel('Soft LTS loss')

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(121, projection='3d')

## Wrapping

### Wrapping function

In [ ]:
from custom_wrapping import wrap
X = np.random.randn(100000)

X_wrap = wrap(X)
X_wrap_rescale = wrap(X, rescale=True)

fig, ax = plt.subplots(1, 1, figsize=(9, 5))
ax.plot(np.sort(X), X_wrap[np.argsort(X)], lw=3)
# ax.plot(np.sort(X), X_wrap_rescale[np.argsort(X)], lw=3)
ax.set_xlabel('Original Data', size=15)
ax.set_ylabel('Wrapped Data', size=15)
_ = plt.xticks(size=12)
_ = plt.yticks(size=12)


### Wrapping regression

In [ ]:
X, _, y, _ = data_simulator.DataSimulatorSimple(
    n_features=1, bad_leverage_perc=0.5, contamination_distance_x=2, contamination_distance_y=2
).get_data(outlier_perc=0.2)
Xy_wrap = custom_wrapping.wrap(np.concatenate((X, y), axis=1))
X_wrap = Xy_wrap[:, 0]
y_wrap = Xy_wrap[:, 1]

In [ ]:
%matplotlib inline
plt.figure(figsize=(15, 6))

plt.scatter(X.flatten()[200:], y.flatten()[200:], alpha=0.5, label='Clean data')
plt.scatter(X.flatten()[:200], y.flatten()[:200], alpha=0.5, color='red', label='Outliers')
plt.scatter(X_wrap, y_wrap, alpha=0.7, c='green', label='$Xy$-wrapped data')
r1 = LinearRegression().fit(X, y)
r2 = LinearRegression().fit(X_wrap[:, None], y_wrap[:, None])
plt.axline([0., r1.predict([[0]])[0][0]], [1., r1.predict([[1]])[0][0]], color='red', lw=3,
           label=f'OLS on original data ($R^2$ on clean data = {r1.score(X[200:], y[200:]):.2f})')
plt.axline([0., r2.predict([[0]])[0][0]], [1., r2.predict([[1]])[0][0]], color='green', lw=3,
           label=f'OLS on $Xy$-wrapped data ($R^2$ on clean data = {r2.score(X[200:], y[200:]):.2f})')
plt.xlabel('X', fontsize=20)
plt.ylabel('y', fontsize=20)
_ = plt.xticks(fontsize=12)
_ = plt.yticks(fontsize=12)
plt.legend(fontsize=12, ncol=3, loc='lower left')

# END